# [과제 2] 회귀분석
### - Ch 1, Ch 2를 토대로 자유롭게 회귀분석과 회귀진단을 진행해주세요.
### - 주석으로 설명 및 근거 자세하게 달아주시면 감사하겠습니다. :)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

# Data

데이터 출처 : https://www.kaggle.com/avikasliwal/used-cars-price-prediction

< y >
* **Price** : The price of the used car in INR Lakhs.



< X >
* Name : The brand and model of the car
* Location : The location in which the car is being sold or is available for purchase.
* Year : The year or edition of the model.
* Kilometers_Driven : The total kilometres driven in the car by the previous owner(s) in KM.
* Fuel_Type : The type of fuel used by the car. (Petrol, Diesel, Electric, CNG, LPG)
* Transmission : The type of transmission used by the car. (Automatic / Manual)
* Owner_Type : Whether the ownership is Firsthand, Second hand or other.
* Mileage : The standard mileage offered by the car company in kmpl or km/kg
* Engine : The displacement volume of the engine in CC.
* Power : The maximum power of the engine in bhp.
* Seats : The number of seats in the car.
* New_Price : The price of a new car of the same model.

In [ ]:
# Load Data
data = pd.read_csv("/content/drive/MyDrive/tobigs/Data/regression_assign2.csv")
data.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [ ]:
data.drop(["Unnamed: 0","New_Price"],axis=1,inplace=True)

In [ ]:
data.describe()

,Year,Kilometers_Driven,Seats,Price
count,6019.000000,6.019000e+03,5977.000000,6019.000000
mean,2013.358199,5.873838e+04,5.278735,9.479468
std,3.269742,9.126884e+04,0.808840,11.187917
min,1998.000000,1.710000e+02,0.000000,0.440000
25%,2011.000000,3.400000e+04,5.000000,3.500000
50%,2014.000000,5.300000e+04,5.000000,5.640000
75%,2016.000000,7.300000e+04,5.000000,9.950000
max,2019.000000,6.500000e+06,10.000000,160.000000


In [ ]:
data.isnull().sum()

Name                  0
Location              0
Year                  0
Kilometers_Driven     0
Fuel_Type             0
Transmission          0
Owner_Type            0
Mileage               2
Engine               36
Power                36
Seats                42
Price                 0
dtype: int64

## Name

In [ ]:
data['Name'].value_counts()

Mahindra XUV500 W8 2WD           49
Maruti Swift VDI                 45
Honda City 1.5 S MT              34
Maruti Swift Dzire VDI           34
Maruti Swift VDI BSIV            31
                                 ..
Ford Fiesta Titanium 1.5 TDCi     1
Mahindra Scorpio S10 AT 4WD       1
Hyundai i20 1.2 Era               1
Toyota Camry W4 (AT)              1
Mahindra Xylo D4 BSIV             1
Name: Name, Length: 1876, dtype: int64

In [ ]:
# 제조사 추출 후, 개수 적은 값은 others 로 대체
other_list = data['Name'].map(lambda x: str(x).strip().split(" ")[0]).value_counts().loc[lambda x : x < 40].index.to_list()

def preprocess_name(name):
    item = ''
    if str(name).split(" ")[0] in other_list:
        item = 'Others'
    else:
        item = str(name).split(" ")[0]
    return item

data['Name'] = data['Name'].apply(preprocess_name)

In [ ]:
data['Name'].value_counts()

Maruti           1211
Hyundai          1107
Honda             608
Toyota            411
Mercedes-Benz     318
Volkswagen        315
Ford              300
Mahindra          272
BMW               267
Audi              236
Tata              186
Skoda             173
Others            158
Renault           145
Chevrolet         121
Nissan             91
Land               60
Jaguar             40
Name: Name, dtype: int64

## Location

In [ ]:
data['Location'].value_counts()

Mumbai        790
Hyderabad     742
Kochi         651
Coimbatore    636
Pune          622
Delhi         554
Kolkata       535
Chennai       494
Jaipur        413
Bangalore     358
Ahmedabad     224
Name: Location, dtype: int64

## Year

In [ ]:
data['Year'].value_counts()

2014    797
2015    744
2016    741
2013    649
2017    587
2012    580
2011    466
2010    342
2018    298
2009    198
2008    174
2007    125
2019    102
2006     78
2005     57
2004     31
2003     17
2002     15
2001      8
2000      4
1998      4
1999      2
Name: Year, dtype: int64

In [ ]:
# 2006보다 작은 연도의 차들은 매우 적음
# 하나로 묶어줌
data['Year']=np.where(data['Year'] < 2006,"less than 2006",data['Year'])

In [ ]:
data['Year'].value_counts()

2014              797
2015              744
2016              741
2013              649
2017              587
2012              580
2011              466
2010              342
2018              298
2009              198
2008              174
less than 2006    138
2007              125
2019              102
2006               78
Name: Year, dtype: int64

## Kilometers Driven

In [ ]:
data['Kilometers_Driven'].value_counts()

60000    82
45000    70
65000    68
50000    61
55000    60
         ..
28937     1
82085     1
68465     1
63854     1
27365     1
Name: Kilometers_Driven, Length: 3093, dtype: int64

## Fuel Type

In [ ]:
data['Fuel_Type'].value_counts()

Diesel      3205
Petrol      2746
CNG           56
LPG           10
Electric       2
Name: Fuel_Type, dtype: int64

In [ ]:
def preprocess_fuel(fuel):
    item = ''
    if fuel not in ['Diesel', 'Petrol']:
        item = 'Others'
    else:
        item = fuel
    return item

data['Fuel_Type'] = data['Fuel_Type'].apply(preprocess_fuel)

In [ ]:
data['Fuel_Type'].value_counts()

Diesel    3205
Petrol    2746
Others      68
Name: Fuel_Type, dtype: int64

## Transmission

In [ ]:
data['Transmission'].value_counts()

Manual       4299
Automatic    1720
Name: Transmission, dtype: int64

## Owner Type

In [ ]:
data['Owner_Type'].value_counts()

First             4929
Second             968
Third              113
Fourth & Above       9
Name: Owner_Type, dtype: int64

In [ ]:
data.loc[data['Owner_Type'] == 'Third', 'Owner_Type'] = 'Third & Above'
data.loc[data['Owner_Type'] == 'Fourth & Above', 'Owner_Type'] = 'Third & Above'

In [ ]:
data['Owner_Type'].value_counts()

First            4929
Second            968
Third & Above     122
Name: Owner_Type, dtype: int64

## Mileage

In [ ]:
data['Mileage'].value_counts()

18.9 kmpl     172
17.0 kmpl     172
18.6 kmpl     119
20.36 kmpl     88
21.1 kmpl      86
             ... 
27.28 kmpl      1
14.57 kmpl      1
22.8 km/kg      1
8.0 kmpl        1
17.24 kmpl      1
Name: Mileage, Length: 442, dtype: int64

In [ ]:
def str2float(mileage):
    item = ''
    item = str(mileage).replace(' kmpl','').replace(' km/kg','')
    item = float(item)
    return item

data['Mileage'] = data['Mileage'].apply(str2float)

In [ ]:
data['Mileage'] = data['Mileage'].fillna(data['Mileage'].mean())

In [ ]:
data['Mileage'].value_counts()

17.00    173
18.90    172
18.60    119
20.36     88
21.10     87
        ... 
9.70       1
14.33      1
27.28      1
14.57      1
17.24      1
Name: Mileage, Length: 431, dtype: int64

## Engine

In [ ]:
data['Engine'].value_counts()

1197 CC    606
1248 CC    512
1498 CC    304
998 CC     259
2179 CC    240
          ... 
2999 CC      1
2147 CC      1
2495 CC      1
3200 CC      1
1797 CC      1
Name: Engine, Length: 146, dtype: int64

In [ ]:
def str2int(engine):
    item = ''
    item = str(engine).replace(' CC','')
    try:
        item = int(item)
    # nan data handling
    except:
        item = 0
    return item

data['Engine'] = data['Engine'].apply(str2int)

In [ ]:
data['Engine'].value_counts()

1197    606
1248    512
1498    304
998     259
2179    240
       ... 
1489      1
2706      1
1978      1
2694      1
1797      1
Name: Engine, Length: 147, dtype: int64

## Power

In [ ]:
data['Power'].value_counts()

74 bhp        235
98.6 bhp      131
73.9 bhp      125
140 bhp       123
78.9 bhp      111
             ... 
76.9 bhp        1
201 bhp         1
199.3 bhp       1
95 bhp          1
181.04 bhp      1
Name: Power, Length: 372, dtype: int64

In [ ]:
def str2float(power):
    item = ''
    item = str(power).replace(' bhp','')
    try:
        item = float(item)
    except:
        item = 0.0
    return item

data['Power'] = data['Power'].apply(str2float)

In [ ]:
data['Power'] = data['Power'].fillna(data['Power'].mean())

In [ ]:
data['Power'].value_counts()

74.00     235
98.60     131
73.90     125
140.00    123
88.50     112
         ... 
201.00      1
199.30      1
95.00       1
152.88      1
181.04      1
Name: Power, Length: 371, dtype: int64

## Seats

In [ ]:
data['Seats'] = data['Seats'].fillna(data['Seats'].mean())

In [ ]:
data['Seats'].value_counts()

5.000000     5014
7.000000      674
8.000000      134
4.000000       99
5.278735       42
6.000000       31
2.000000       16
10.000000       5
9.000000        3
0.000000        1
Name: Seats, dtype: int64

## Encoding

In [ ]:
data.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti,Mumbai,2010,72000,Others,Manual,First,26.60,998,58.16,5.0,1.75
1,Hyundai,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.20,5.0,12.50
2,Honda,Chennai,2011,46000,Petrol,Manual,First,18.20,1199,88.70,5.0,4.50
3,Maruti,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7.0,6.00
4,Audi,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968,140.80,5.0,17.74


In [ ]:
data.describe()

,Kilometers_Driven,Mileage,Engine,Power,Seats,Price
count,6.019000e+03,6019.000000,6019.000000,6019.000000,6019.000000,6019.000000
mean,5.873838e+04,18.134961,1611.579498,111.227632,5.278735,9.479468
std,9.126884e+04,4.581528,612.449869,55.294789,0.806012,11.187917
min,1.710000e+02,0.000000,0.000000,0.000000,0.000000,0.440000
25%,3.400000e+04,15.170000,1197.000000,74.000000,5.000000,3.500000
50%,5.300000e+04,18.150000,1493.000000,93.700000,5.000000,5.640000
75%,7.300000e+04,21.100000,1969.000000,138.030000,5.000000,9.950000
max,6.500000e+06,33.540000,5998.000000,560.000000,10.000000,160.000000


In [ ]:
data.isnull().sum()

Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
lst = ['Name','Location','Fuel_Type','Transmission','Owner_Type','Year']
for i in lst:
    le = LabelEncoder()
    data[i]=le.fit_transform(data[i])

In [ ]:
data.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,9,9,4,72000,1,1,0,26.60,998,58.16,5.0,1.75
1,5,10,9,41000,0,1,0,19.67,1582,126.20,5.0,12.50
2,4,2,5,46000,2,1,0,18.20,1199,88.70,5.0,4.50
3,9,2,6,87000,0,1,0,20.77,1248,88.76,7.0,6.00
4,0,3,7,40670,0,0,1,15.20,1968,140.80,5.0,17.74


# Training

In [ ]:
X = data.drop("Price",axis=1)
Y= data['Price']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, Y_train)
y_pred= model.predict(X_test)
print("Accuracy on Traing set: ",model.score(X_train,Y_train))
print("Accuracy on Testing set: ",model.score(X_test,Y_test))

Accuracy on Traing set:  0.6862203843249046
Accuracy on Testing set:  0.2571098545790358
